# 크롤링
selenium과 beautifulsoup을 통해 *데이터*를 뽑아보자!

In [1]:
!pip install selenium # 셀레니움 설치

Invalid requirement: '#'
Traceback (most recent call last):
  File "c:\python\anaconda3\lib\site-packages\pip\_vendor\packaging\requirements.py", line 93, in __init__
    req = REQUIREMENT.parseString(requirement_string)
  File "c:\python\anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1632, in parseString
    raise exc
  File "c:\python\anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1622, in parseString
    loc, tokens = self._parse( instring, 0 )
  File "c:\python\anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1379, in _parseNoCache
    loc,tokens = self.parseImpl( instring, preloc, doActions )
  File "c:\python\anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 3395, in parseImpl
    loc, exprtokens = e._parse( instring, loc, doActions )
  File "c:\python\anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1379, in _parseNoCache
    loc,tokens = self.parseImpl( instring, preloc, doActions )
  File "c:\python\anaconda3\lib\site-p

In [2]:
from urllib import request
from selenium import webdriver

driver = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')

# selenium을 이용해 html 뽑기

**driver.get('url')** # url 은 들어가고 싶은 사이트의 주소  
**driver.find_element_by_xpath('').click()** # 사이트 내에서 클릭해서 들어가야할 때 쓰는 코드 # html에서 마우스 왼쪽을 눌러 copy-xpath로 가져오기  

In [3]:
# 방송사들어가기
url=driver.get('http://www.ebs.co.kr/tv/program')

In [4]:
# 분류 버튼 클릭하기
driver.find_element_by_xpath('//*[@id="Contents"]/div/div/div[2]/div[2]/ul[1]/li[2]').click()

In [5]:
# 원하는 소분류(=다큐멘터리) 클릭하기()
driver.find_element_by_xpath('//*[@id="Contents"]/div/div/div[3]/ul/li[2]/ul/li[2]/label').click()

In [6]:
# 리스트 클릭하기
driver.find_element_by_xpath('//*[@id="Contents"]/div/div/div[2]/div[2]/ul[2]/li[2]/a[2]').click()

In [7]:
# 원하는 소분류클릭하기(=vod)
driver.find_element_by_xpath('//*[@id="tabVod"]').click()

# 페이지 내리기

In [8]:
SCROLL_PAUSE_TIME = 0.1
import time 
# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        continue
    last_height = new_height
break

KeyboardInterrupt: 

### 클릭은 셀레니움으로 했고
## 텍스트 끌어오는 것은 beautifulsoup으로 가져온다.
가져와야하는 일
* link
* title
* episode
* date
* synopsis

In [9]:
from bs4 import BeautifulSoup

html = driver.page_source # 현재 페이지 주소를 반환 (중.요!!)
soup = BeautifulSoup(html,'html.parser')

# 링크가져오기

In [43]:
# 범위를 주고 test해보기
i = 1
for i in range(1,10):
    links = driver.find_elements_by_css_selector('#layerProgram_List_vod > li:nth-child('+str(i)+') > div > strong > a')
    urls = [ i.get_attribute('href') for i in links]
#    print(urls)

In [39]:
# 본 실습
page = 1
all_links = []
for page in range(1,12500):
    links = driver.find_elements_by_css_selector('#layerProgram_List_vod > li:nth-child('+str(page)+') > div > strong > a')
    for i in links:
        urls=i.get_attribute('href')
    page = page +1
#     print(urls)
    all_links.append(urls)
# all_links
# 뽑은 데이터는 리스트 형식으로 만들어주자.

KeyboardInterrupt: 

In [ ]:
import pandas as pd

data = pd.DataFrame(
    {'link': all_links },
    columns={'link'})
data

In [ ]:
data.to_csv('links.csv',sep=',', encoding='utf-8')

# vod title만 가져오기

In [17]:
list_all = soup.find_all('strong','mainTit')
#list_all

In [18]:
send_list = [list_all[n].find('a').get_text() for n in range(0, len(list_all))] # a가 있는 텍스트 모두 가져오기 = vod title
#send_list

In [13]:
len(send_list) # 길이확인

12467

In [19]:
import pandas as pd

data2 = pd.DataFrame(
    {'title': send_list},
    columns={'title'})
data2.head()

,title
0,지식채널e
1,다큐 시선
2,한국기행
3,세계테마기행
4,세계의 눈


In [15]:
data2.to_csv('title.csv',sep=',', encoding='utf-8')

# episode 가져오기

In [20]:
name_all = soup.find_all('span','tbl_td col3')
#name_all

In [21]:
len(name_all) # 길이확인

12467

In [ ]:
# name_list = [name_all[n].find('a').get_text() for n in range(0, len(name_all))]
# name_list

In [24]:
titles=[]
for e in name_all:
    titles.append(e.text)
#titles

In [25]:
for i in titles:
#    print(i.strip())
# 공백제거

SyntaxError: unexpected EOF while parsing (<ipython-input-25-36f529a98738>, line 3)

In [27]:
tit=[]
for i in titles:
    tit.append(i.strip())
#tit

In [42]:
import pandas as pd

data3 = pd.DataFrame(
    {'title':tit,'episode': send_list},
    columns={'title','episode'})
#data3.tail()

In [29]:
data3.to_csv('epi_title.csv',sep=',', encoding='utf-8')

# 날짜 가져오기

In [31]:
day_all = soup.find_all('span','tbl_td col2')
#day_all

In [33]:
days=[]
for d in day_all:
    days.append(d.text)
#days

In [41]:
import pandas as pd

data4 = pd.DataFrame(
    {'title':tit,'date':days,'episode': send_list},
    columns={'title','date','episode'})
#data4[:50]

In [36]:
data4.to_csv('date_ti_epi_link.csv',sep=',', encoding='utf-8')

## 뽑아온 데이터를 한 눈에 보자

In [37]:
import pandas as pd

In [40]:
df = pd.read_csv('result.csv', encoding='cp949')
df.shape
#df.head()

(12478, 7)

# synopsis 뽑기
시놉시스를 뽑기 위해서는 링크를 가져와서 자동화해야한다.

In [ ]:
df['링크'][0] # 첫 번째 링크 확인

In [ ]:
from urllib import request
from selenium import webdriver

driver = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')

In [ ]:
from bs4 import BeautifulSoup

html = driver.page_source # 현재 페이지 주소를 반환 (중.요!!)
soup = BeautifulSoup(html,'html.parser')

In [ ]:
import time
all_synopsis=[]
for i in range(0,20): # 범위를 지정해서 조금씩 뽑아낸다.
    i=i+1
#     print(df['링크'][i])
    driver.get(df['링크'][i])
    time.sleep(1)
    html = driver.page_source # 현재 페이지 주소를 반환 (중.요!!)
    soup = BeautifulSoup(html,'html.parser')
    for i in soup.select('p.detail_story'):
        synopsis=i.text
    all_synopsis.append(synopsis)
all_synopsis
# 자동화 개선 필요함

In [ ]:
import pandas as pd

test = pd.DataFrame(
    {'synopsis': all_synopsis },
    columns={'synopsis'})
test.tail()

In [ ]:
test.to_csv('synop_test.csv',sep=',', encoding='utf-8') # 저장하기

In [ ]:
df=pd.read_csv('synop_test.csv',encoding='cp949') # 불러오기
df.head()

### 시놉시스 전처리
시놉시스 끝에 <펼쳐보기>라고 써져있는 text 제거  
공백 제거  

In [ ]:
df=df.replace("펼쳐보기","") 
df.head()

In [ ]:
synop=df['시놉시스']

synop
sn=[]
for i in synop:
    sn.append(i.strip())
sn

In [ ]:
import pandas as pd

data6 = pd.DataFrame(
    { 'synop': sn},
    columns={'synop'})
data6.tail()

#  확인하기

In [ ]:
df['제목'].value_counts()

# 느낀점
크롤링 시간이 너무 오래걸림  
위에서 아래까지 다 가져오고 업데이트 된 부분은 수기로 적어주는게 더 빠를 듯  
한 번 크롤링 잘해서 나중에 고생을 덜 하자.  